## Drug Molecule Generation with VAE

**Author:** [Victor Basu](https://www.linkedin.com/in/victor-basu-520958147)<br>
**Date created:** 2022/03/10<br>
**Last modified:** 2022/03/27<br>
**Description:** Implementing a Convolutional Variational AutoEncoder (VAE) for Drug Discovery.

In [23]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

#import ast
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Lambda

In [24]:
from tensorflow.keras import layers
import pandas as pd

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from rdkit import Chem, RDLogger
from rdkit.Chem import BondType
from rdkit.Chem.Draw import MolsToGridImage
from rdkit.Chem import Draw
RDLogger.DisableLog("rdApp.*")

In [25]:
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))
print("GPU in use:", tf.test.gpu_device_name())

TensorFlow version: 2.13.0
GPU available: []
GPU in use: 


In [26]:
df = pd.read_csv('dataset1.csv')
df.drop([0,1,2,3,4], inplace=True)
df=df.rename(columns = {'PUBCHEM_EXT_DATASOURCE_SMILES':'SMILES','PUBCHEM_ACTIVITY_OUTCOME':'Activity', 'PUBCHEM_ACTIVITY_SCORE':'Score'})
columns_to_drop = [col for col in df.columns if col not in ['SMILES', 'Activity', 'Score', 'Potency', 'Efficacy']]
df = df.drop(columns = columns_to_drop)
#df=df.drop(['Unnamed: 3','Unnamed: 4','Unnamed: 5'], axis=1)
df = df.dropna(subset=['SMILES'])

df=df.fillna(0)
df.head()

,SMILES,Activity,Score,Potency,Efficacy
5,CNCC1=NC2=C(C=C(C=C2)Cl)C(=N1)C3=CC=CN3,Inactive,0.0,0.0,0.0
6,CCSC(=NC1=CC=C(C=C1)C(F)(F)F)N.Cl,Inactive,0.0,0.0,0.0
7,CCN(CC1=CC(=CC=C1)S(=O)(=O)[O-])C2=CC=C(C=C2)C...,Inactive,0.0,0.0,0.0
8,CC1=CC=C(C=C1)S(=O)(=O)N2CCN(CC2)C3=NC(=NC4=CC...,Inactive,0.0,0.0,0.0
9,CC1=CC=C(C=C1)S(=O)(=O)N2CCN(CC2)C3=NC(=NC4=CC...,Inactive,0.0,0.0,0.0


In [27]:
#This part should now be 6000 inactive + 6000 active 
smiles = df['SMILES'].tolist()
small_molecules = []
for smile in smiles:
    mol = Chem.MolFromSmiles(smile)
    if mol.GetNumAtoms() <= 50:
        small_molecules.append(smile)
df_sampled = df[df['SMILES'].isin(small_molecules)].reset_index(drop=True)

In [28]:
filtered_df = df_sampled.sample(frac=0.015, random_state=42)
filtered_df

,SMILES,Activity,Score,Potency,Efficacy
271139,C1CCC(CC1)C2=CC=C(C=C2)S(=O)(=O)N3CCN(CC3)C4=C...,Inactive,0.0,0.0,0.0
301775,COC1=CC=C(C=C1)CNC(=O)CSC2=NC3=CC=CC=C3N=C2CC4...,Inactive,0.0,0.0,0.0
124783,C1OC2=C(O1)C=C(C=C2)NC(=O)CSC3=NC=NC4=CC=CC=C43,Inactive,0.0,0.0,0.0
56239,CC1CC2=C(N1C(=O)C)C=CC(=C2)S(=O)(=O)N3CCC(CC3)...,Inactive,0.0,0.0,0.0
75807,C=CCN1C(=NNC1=S)C(C2=CC=CC=C2)C3=CC=CC=C3,Inactive,0.0,0.0,0.0
...,...,...,...,...,...
227591,CC(C)(C)CC(=O)NCC(C1=CC2=C(C=C1)OCO2)N3CCN(CC3...,Inactive,0.0,0.0,0.0
74238,CC1=CC(=C(C=C1)C)NS(=O)(=O)C2=CC=C(C=C2)OCC(=O...,Inactive,0.0,0.0,0.0
175953,CCC(=O)C1=CC=C(C=C1)OCC(=O)NC2CCCC2,Inactive,0.0,0.0,0.0
252727,CCOC1=C(C=CC(=C1)/C=C/2\C(=O)N(C(=NC3=CC=CC=C3...,Inactive,0.0,0.0,0.0


In [29]:
from rdkit import Chem
from rdkit.Chem import rdmolops

In [31]:
smiles = filtered_df['SMILES'].tolist()

## Molecular Representation

In [32]:
search_elements=[]
for smile in smiles:
    mol = Chem.MolFromSmiles(smile)
    atoms = list(set([atom.GetSymbol() for atom in mol.GetAtoms()]))
    search_elements += atoms
    search_elements = list(set(search_elements))

print(search_elements)

['P', 'O', 'Na', 'Cl', 'Br', 'S', 'K', 'F', 'N', 'I', 'C', 'B']


In [33]:
bond_mapping = {"SINGLE": 0, "DOUBLE": 1, "TRIPLE": 2, "AROMATIC": 3}
bond_mapping.update(
    {0: BondType.SINGLE, 1: BondType.DOUBLE, 2: BondType.TRIPLE, 3: BondType.AROMATIC}
)

In [34]:
MAX_MOLSIZE = max(filtered_df['SMILES'].str.len())
SMILE_to_index = dict((c, i) for i, c in enumerate(SMILE_CHARSET))
index_to_SMILE = dict((i, c) for i, c in enumerate(SMILE_CHARSET))
atom_mapping = dict(SMILE_to_index)
atom_mapping.update(index_to_SMILE)
print(atom_mapping)
print("Max molecule size: {}".format(MAX_MOLSIZE))
print("Character set Length: {}".format(len(SMILE_CHARSET)))

{'P': 0, 'O': 1, 'Na': 2, 'Cl': 3, 'Br': 4, 'S': 5, 'K': 6, 'F': 7, 'N': 8, 'I': 9, 'C': 10, 'B': 11, 0: 'P', 1: 'O', 2: 'Na', 3: 'Cl', 4: 'Br', 5: 'S', 6: 'K', 7: 'F', 8: 'N', 9: 'I', 10: 'C', 11: 'B'}
Max molecule size: 119
Character set Length: 12


In [35]:
NUM_ATOMS = 50 #Max number of atoms
ATOM_DIM = len(SMILE_CHARSET)  # Number of atom types
BOND_DIM = 5 # Number of bond types

In [36]:
 def smiles_to_graph(smiles):
    # Converts SMILES to molecule object
    molecule = Chem.MolFromSmiles(smiles)

    # Initialize adjacency and feature tensor
    adjacency = np.zeros((BOND_DIM, NUM_ATOMS, NUM_ATOMS), "float32")
    features = np.zeros((NUM_ATOMS, ATOM_DIM), "float32")

    # Bond weights
    bond_weights = {
        "SINGLE": 1.0,
        "DOUBLE": 2.0,
        "TRIPLE": 3.0,
        "AROMATIC": 2.5}

    # Loop over each atom in molecule
    for atom in molecule.GetAtoms():
        i = atom.GetIdx()
        atom_type = atom_mapping[atom.GetSymbol()]
        features[i] = np.eye(ATOM_DIM)[atom_type]

        # Loop over one-hop neighbors
        for neighbor in atom.GetNeighbors():
            j = neighbor.GetIdx()
            bond = molecule.GetBondBetweenAtoms(i, j)
            bond_type = bond.GetBondType().name
            bond_weight = bond_weights.get(bond_type, 1.0)  

            bond_type_idx = bond_mapping[bond_type]
            adjacency[bond_type_idx, [i, j], [j, i]] = bond_weight

    # Where no bond, add 1 to last channel (indicating "non-bond")
    # Notice: channels-first
    adjacency[-1, np.sum(adjacency, axis=0) == 0] = 1

    # Where no atom, add 1 to last column (indicating "non-atom")
    features[np.where(np.sum(features, axis=1) == 0)[0], -1] = 1

    return adjacency, features


def graph_to_molecule(adjacency, features):

    # RWMol is a molecule object intended to be edited
    molecule = Chem.RWMol()

    # Remove "no atoms" & atoms with no bonds
    keep_idx = np.where(
        (np.argmax(features, axis=1) != ATOM_DIM - 1)
        & (np.sum(adjacency[:-1], axis=(0, 1)) != 0))[0]

    features = features[keep_idx]
    adjacency = adjacency[:, keep_idx, :][:, :, keep_idx]

    # Add atoms to molecule
    for atom_type_idx in np.argmax(features, axis=1):
        atom = Chem.Atom(atom_mapping[atom_type_idx])
        _ = molecule.AddAtom(atom)

    added_bonds = set()
    (bonds_ij, atoms_i, atoms_j) = np.where(np.triu(adjacency) == 1)
    for (bond_ij, atom_i, atom_j) in zip(bonds_ij, atoms_i, atoms_j):
        if atom_i == atom_j or bond_ij == BOND_DIM - 1:
            continue
        bond_type = bond_mapping[bond_ij]
        if (atom_i, atom_j) in added_bonds or (atom_j, atom_i) in added_bonds:
            
            continue
        molecule.AddBond(int(atom_i), int(atom_j), bond_type)
        added_bonds.add((atom_i, atom_j))

    # Sanitize the molecule; for more information on sanitization, see
    # https://www.rdkit.org/docs/RDKit_Book.html#molecular-sanitization
    flag = Chem.SanitizeMol(molecule, catchErrors=True)
    if flag != Chem.SanitizeFlags.SANITIZE_NONE:
        return None

    return molecule

##  Generate training set

In [37]:
'''
    Directly from Reference: https://keras.io/examples/generative/wgan-graphs/
'''
class RelationalGraphConvLayer(keras.layers.Layer):
    def __init__(
        self,
        units=128,
        activation="relu",
        use_bias=False,
        kernel_initializer="glorot_uniform",
        bias_initializer="zeros",
        kernel_regularizer=None,
        bias_regularizer=None,
        **kwargs
    ):
        super().__init__(**kwargs)

        self.units = units
        self.activation = keras.activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.bias_initializer = keras.initializers.get(bias_initializer)
        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)
        self.bias_regularizer = keras.regularizers.get(bias_regularizer)

    def build(self, input_shape):
        bond_dim = input_shape[0][1]
        atom_dim = input_shape[1][2]

        self.kernel = self.add_weight(
            shape=(bond_dim, atom_dim, self.units),
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            trainable=True,
            name="W",
            dtype=tf.float32,
        )

        if self.use_bias:
            self.bias = self.add_weight(
                shape=(bond_dim, 1, self.units),
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                trainable=True,
                name="b",
                dtype=tf.float32,
            )

        self.built = True

    def call(self, inputs, training=False):
        adjacency, features = inputs
        # Aggregate information from neighbors
        x = tf.matmul(adjacency, features[:, None, :, :])
        # Apply linear transformation
        x = tf.matmul(x, self.kernel)
        if self.use_bias:
            x += self.bias
        # Reduce bond types dim
        x_reduced = tf.reduce_sum(x, axis=1)
        # Apply non-linear transformation
        return self.activation(x_reduced)

## Build the Encoder and Decoder
The Encoder takes as input a molecule's graph adjacency matrix and feature matrix.
These features are processed via a Graph Convolution layer, then are flattened and
processed by several Dense layers to derive `z_mean` and `log_var`, the
latent-space representation of the molecule.
**Graph Convolution layer**: The relational graph convolution layer implements
non-linearly transformed neighbourhood aggregations. We can define these layers as
follows:
`H_hat**(l+1) = σ(D_hat**(-1) * A_hat * H_hat**(l+1) * W**(l))`
Where `σ` denotes the non-linear transformation (commonly a ReLU activation), `A` the
adjacency tensor, `H_hat**(l)` the feature tensor at the `l-th` layer, `D_hat**(-1)` the
inverse diagonal degree tensor of `A_hat`, and `W_hat**(l)` the trainable weight tensor
at the `l-th` layer. Specifically, for each bond type (relation), the degree tensor
expresses, in the diagonal, the number of bonds attached to each atom.
Source:
[WGAN-GP with R-GCN for the generation of small molecular graphs](https://keras.io/examples/generative/wgan-graphs/))
The Decoder takes as input the latent-space representation and predicts
the graph adjacency matrix and feature matrix of the corresponding molecules.

In [66]:
class SymLayer(layers.Layer):
    def call(self,x):
        return (x + tf.transpose(x, (0,1,3,2,))) / 2

In [67]:
def get_encoder(gconv_units, latent_dim, adjacency_shape, feature_shape, dense_units, dropout_rate, regularizer=None):
    adjacency = keras.layers.Input(shape=adjacency_shape, name="adjacency_input")
    features = keras.layers.Input(shape=feature_shape, name="feature_input")
    scores = keras.layers.Input(shape=(1,), name="score_input")  # Conditional input (scalar)

    # Graph convolution layers
    features_transformed = features
    for units in gconv_units:
        features_transformed = RelationalGraphConvLayer(units)(
            [adjacency, features_transformed]
        )

    # Reduce 2D representation to 1D
    x = keras.layers.GlobalAveragePooling1D()(features_transformed)

    # Concatenate the score (condition) to the reduced graph representation
    x = keras.layers.Concatenate()([x, scores])

    # Fully connected layers
    for units in dense_units:
        x = layers.Dense(units, activation="relu", kernel_regularizer=regularizer)(x)
        x = layers.Dropout(dropout_rate)(x)

    # Latent space
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)

    # Create encoder model
    encoder = keras.Model(inputs=[adjacency, features, scores], outputs=[z_mean, z_log_var], name="encoder")

    return encoder


def get_decoder(dense_units, latent_dim, adjacency_shape, feature_shape, dropout_rate, regularizer=None):
    latent_input = keras.Input(shape=(latent_dim,), name="latent_input")
    scores = keras.Input(shape=(1,), name="score_input")  # Conditional input (scalar)

    # Concatenate latent input with the conditional score
    x = keras.layers.Concatenate()([latent_input, scores])

    # Dense layers
    for units in dense_units:
        x = keras.layers.Dense(units, activation="tanh", kernel_regularizer=regularizer)(x)
        x = keras.layers.Dropout(dropout_rate)(x)

    # Adjacency reconstruction
    adj_output = keras.layers.Dense(tf.math.reduce_prod(adjacency_shape).numpy().astype(int))(x)
    adj_output = keras.layers.Reshape(adjacency_shape)(adj_output)
    print(type(adj_output))
    adj_output = SymLayer()(adj_output)
    #adj_output = Lambda(lambda x: (x + tf.transpose(x, (0, 1, 3, 2))) / 2)(adj_output)
    print(type(adj_output))

    # Feature reconstruction
    feat_output = keras.layers.Dense(tf.math.reduce_prod(feature_shape).numpy().astype(int))(x)
    feat_output = keras.layers.Reshape(feature_shape)(feat_output)
    feat_output = keras.layers.Softmax(axis=2)(feat_output)

    # Create decoder model
    decoder = keras.Model(inputs=[latent_input, scores], outputs=[adj_output, feat_output], name="decoder")

    return decoder

## Build the VAE
This model is trained to optimize four losses:
* Categorical crossentropy
* KL divergence loss
* Property prediction loss
* Graph loss (gradient penalty)
The categorical crossentropy loss function measures the model's
reconstruction accuracy. The Property prediction loss estimates the mean squared
error between predicted and actual properties after running the latent representation
through a property prediction model. The property
prediction of the model is optimized via binary crossentropy. The gradient
penalty is further guided by the model's property (SAS) prediction.
A gradient penalty is an alternative soft constraint on the
1-Lipschitz continuity as an improvement upon the gradient clipping scheme from the
original neural network
("1-Lipschitz continuity" means that the norm of the gradient is at most 1 at evey single
point of the function).
It adds a regularization term to the loss function.

In [53]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, beta=1.0, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def call(self, inputs):
        adjacency, features, scores = inputs
        z_mean, z_log_var = self.encoder([adjacency, features, scores])
        z = self.reparameterize(z_mean, z_log_var)
        return self.decoder([z, scores])
        
    def sampling(self, args):
        """
        Reparameterization trick: Sample from a Gaussian distribution using
        z = z_mean + epsilon * exp(z_log_var / 2), where epsilon is sampled from N(0, 1).
        """
        z_mean, z_log_var = args
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))  # Standard normal noise
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [54]:
train, test = train_test_split(filtered_df,test_size=0.2,random_state=42)

train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)

train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

adj_train, fea_train, score_train = [], [], []
adj_val, fea_val, score_val = [], [], []

for idx in range(len(train_df)):
    adjacency, features = smiles_to_graph(train_df.loc[idx]["SMILES"])
    score = train_df.loc[idx]["Score"]
    adj_train.append(adjacency)
    fea_train.append(features)
    score_train.append(score)

for idx in range(len(val_df)):
    adjacency, features = smiles_to_graph(val_df.loc[idx]["SMILES"])
    score = val_df.loc[idx]["Score"]
    adj_val.append(adjacency)
    fea_val.append(features)
    score_val.append(score)

    
adj_train = np.array(adj_train)/3
fea_train = np.array(fea_train)
score_train = np.array(score_train).reshape(-1,1)

adj_val = np.array(adj_val)/3
fea_val = np.array(fea_val)
score_val = np.array(score_val).reshape(-1,1)

print(adj_train.shape)
print(fea_train.shape)
print(score_train.shape)
print(adj_val.shape)
print(fea_val.shape)
print(score_val.shape)

(3276, 5, 50, 50)
(3276, 50, 12)
(3276, 1)
(819, 5, 50, 50)
(819, 50, 12)
(819, 1)


In [55]:
print(adj_train[100])

[[[0.         0.33333334 0.         ... 0.         0.         0.        ]
  [0.33333334 0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.      

## Train the model

In [56]:
#Hyperparameters
batch_size = 24
EPOCHS = 10
VAE_LR = 2e-4
LATENT_DIM = 16  # Size of the latent space

In [68]:
from tensorflow.keras.regularizers import l1_l2
encoder = get_encoder(
    gconv_units=[9],
    adjacency_shape=(BOND_DIM, NUM_ATOMS, NUM_ATOMS),
    feature_shape=(NUM_ATOMS, ATOM_DIM),
    latent_dim=LATENT_DIM,
    dense_units=[256, 512],
    dropout_rate=0,
    regularizer=l1_l2(l1=1e-3, l2=1e-3)
)
decoder = get_decoder(
    dense_units=[256, 512, 1024],
    dropout_rate=0.2,
    latent_dim=LATENT_DIM,
    adjacency_shape=(BOND_DIM, NUM_ATOMS, NUM_ATOMS),
    feature_shape=(NUM_ATOMS, ATOM_DIM), 
    regularizer=l1_l2(l1=0.1, l2=0.1)
)
vae = VAE(encoder, decoder)

vae.compile(optimizer=keras.optimizers.Adam(learning_rate=VAE_LR))

#Normalize score matrix
min_score = np.min(score_train)
max_score = np.max(score_train) 
score_train_normalized = (score_train - min_score) / (max_score - min_score)


<class 'keras.src.engine.keras_tensor.KerasTensor'>
<class 'keras.src.engine.keras_tensor.KerasTensor'>


In [44]:
train_dataset = tf.data.Dataset.from_tensor_slices((adj_train, fea_train, score_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((adj_val, fea_val, score_val)).batch(batch_size)

val_loss_list = []
train_loss_list = []

#These starting values are arbitary. We picked those starting values because our loss is constantly lower than 100.
#We picked reconstruction_loss < prev to lower the beta first, because we suspected there is a reconstruction issue
prev_rec_loss = 100
reconstruction_loss = 101
beta = 0.05

In [45]:
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    
    if reconstruction_loss < prev_rec_loss * 0.99:
        beta = max(beta * 0.9, 0.001)  
    else:  
        beta = min(beta * 1.1, 0.1)
    prev_rec_loss = reconstruction_loss
    
    # Training Loop
    train_loss = 0
    for step, (adjacency, features, scores) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass
            z_mean, z_log_var = vae.encoder([adjacency, features, scores])
            z = vae.sampling([z_mean, z_log_var])
            adj_reconstruction, feature_reconstruction = vae.decoder([z, scores])

            # Compute losses
            adj_loss = tf.reduce_mean(
                tf.reduce_sum(keras.losses.binary_crossentropy(adjacency, adj_reconstruction), axis=(1, 2))
            )
            feat_loss = tf.reduce_mean(
                tf.reduce_sum(keras.losses.categorical_crossentropy(features, feature_reconstruction), axis=1)
            )
            reconstruction_loss = 0.6*adj_loss + 0.4*feat_loss
            kl_loss = -0.5 * tf.reduce_mean(
                tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)
            )
            total_loss = reconstruction_loss + beta * kl_loss

        # Backpropagation
        grads = tape.gradient(total_loss, vae.trainable_weights)
        vae.optimizer.apply_gradients(zip(grads, vae.trainable_weights))

        train_loss += total_loss

    train_loss /= len(train_dataset)
    train_loss_list.append(train_loss)
    print(f"Train Loss: {train_loss.numpy()}, KL Loss: {kl_loss.numpy()}, Reconstruction Loss: {reconstruction_loss.numpy()}")

    # Validation Loop
    val_loss = 0
    for val_step, (val_adjacency, val_features, val_scores) in enumerate(val_dataset):
        # Forward pass
        z_mean, z_log_var = vae.encoder([val_adjacency, val_features, val_scores])
        z = vae.sampling([z_mean, z_log_var])
        val_adj_reconstruction, val_feat_reconstruction = vae.decoder([z, val_scores])

        # Compute losses
        val_adj_loss = tf.reduce_mean(
            tf.reduce_sum(keras.losses.binary_crossentropy(val_adjacency, val_adj_reconstruction), axis=(1, 2))
        )
        val_feat_loss = tf.reduce_mean(
            tf.reduce_sum(keras.losses.categorical_crossentropy(val_features, val_feat_reconstruction), axis=1)
        )
        val_reconstruction_loss = 0.6*val_adj_loss + 0.4*val_feat_loss
        val_kl_loss = -0.5 * tf.reduce_mean(
            tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)
        )
        val_total_loss = val_reconstruction_loss + beta * val_kl_loss

        val_loss += val_total_loss

    val_loss /= len(val_dataset)
    val_loss_list.append(val_loss)
    print(f"Validation Loss: {val_loss.numpy()}, KL Loss: {val_kl_loss.numpy()}, Reconstruction Loss: {val_reconstruction_loss.numpy()}")
    print('BETA is: ', beta)

Epoch 1/10
Train Loss: 61.168907165527344, KL Loss: 7.120011806488037, Reconstruction Loss: 50.62772750854492
Validation Loss: 50.37357711791992, KL Loss: 7.551031112670898, Reconstruction Loss: 49.380767822265625
BETA is:  0.05500000000000001
Epoch 2/10
Train Loss: 50.16172409057617, KL Loss: 6.893106937408447, Reconstruction Loss: 49.703346252441406
Validation Loss: 49.048683166503906, KL Loss: 7.292844295501709, Reconstruction Loss: 47.89186477661133
BETA is:  0.04950000000000001
Epoch 3/10
Train Loss: 48.23603057861328, KL Loss: 8.59839153289795, Reconstruction Loss: 48.40557098388672
Validation Loss: 47.653785705566406, KL Loss: 9.818497657775879, Reconstruction Loss: 46.010799407958984
BETA is:  0.044550000000000006
Epoch 4/10
Train Loss: 47.532657623291016, KL Loss: 9.545109748840332, Reconstruction Loss: 48.02184295654297
Validation Loss: 47.084415435791016, KL Loss: 11.15727424621582, Reconstruction Loss: 45.52397537231445
BETA is:  0.040095000000000006
Epoch 5/10
Train Loss: 

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(EPOCHS), train_loss_list, label='Training Loss', marker='o')
plt.plot(range(EPOCHS), val_loss_list, label='Validation Loss', marker='s')

# Add title and labels
plt.title('Training and Validation Loss Over Epochs', fontsize=16)
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.legend()
plt.show()

## Model Inferencing

We would be inferring our model to predict over random latent space and try to generate 100 new valid molecules.

### Generate unique Molecules with the model

In [ ]:
from datetime import datetime
# Create output directory if it doesn't exist
output_dir = "generated_molecules"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Get current timestamp for unique folder name
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_dir = os.path.join(output_dir, f"run_{timestamp}")
os.makedirs(run_dir)

valid_count = 0
iterations = 100
for i in range(iterations):
    
    # Random latent vector and target score
    random_z = np.random.normal(size=(1, LATENT_DIM))
    target_score = np.array([[0.5]]) 

    # Generate predictions
    adjacency_pred, feature_pred = vae.decoder.predict([random_z, target_score])
    adjacency_pred = adjacency_pred*3 #Denormalize adjacency matrix
    # Post-process predictions
    threshold = 0.5
    adjacency_pred_processed = (adjacency_pred > threshold).astype(int)  # Binary
    adjacency_pred_processed = np.maximum(
        adjacency_pred_processed, 
        np.transpose(adjacency_pred_processed, (0, 1, 3, 2))  # Swap last two dimensions
    )
    feature_pred_processed = tf.one_hot(np.argmax(feature_pred, axis=-1), depth=ATOM_DIM).numpy()  # One-hot

    # Debug prints
    #print("Processed Adjacency Matrix Shape:", adjacency_pred_processed[0].shape)
    #print("Processed Feature Matrix Shape:", feature_pred_processed[0].shape)

    # Generate the molecule
    generated_molecule = graph_to_molecule(adjacency_pred_processed[0], feature_pred_processed[0])

    # Visualize or print the molecule
    if generated_molecule:
        valid_count += 1
        smiles = Chem.MolToSmiles(generated_molecule)
        
        # Create filename using molecule index and SMILES string (truncated if too long)
        truncated_smiles = smiles[:50]  # Truncate SMILES to prevent filename length issues
        safe_smiles = "".join(c if c.isalnum() else "_" for c in truncated_smiles)  # Remove special characters
        filename = f"molecule_{valid_count:04d}_{safe_smiles}"
        
        # Save the molecule image
        img_path = os.path.join(run_dir, f"{filename}.png")
        Draw.MolToFile(generated_molecule, img_path, size=(300, 300))
        
        # Save SMILES string to a text file
        with open(os.path.join(run_dir, "molecules.txt"), "a") as f:
            f.write(f"Molecule {valid_count}: {smiles}\n")
            
    else:
        print("Failed to generate a valid molecule.")

print(f"\nGeneration complete! Generated {valid_count} valid molecules out of {iterations} runs.")
#print(f"Output directory: {run_dir}")